In [ ]:
import numpy as np
import scanpy as sc
import numpy as np
import pandas as pd

import anndata2ri
import logging
from scipy.sparse import issparse
from CSCORE.CSCORE_IRLS import CSCORE_IRLS

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

In [ ]:
rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R
library(Seurat)
library(sctransform)
library(Hmisc)

In [ ]:
adata = sc.read_h5ad(f"scatac_heart_674.h5ad")
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata.var_names = [i.split('.')[0] for i in adata.var_names]

In [ ]:
import mygene
mg = mygene.MyGeneInfo()
geneList = adata.var_names
geneSyms = mg.querymany(geneList , scopes='ensembl.gene', fields='symbol', species='human')

In [ ]:
gene_set = {}
for i in geneSyms:
    if 'symbol' in i.keys():
        gene_set[i['query']] = i['symbol'] 
adata = adata[:, list(gene_set.keys())]
adata.var_names = [gene_set[i] for i in adata.var_names]
adata.var['mt'] = adata.var_names.str.startswith('MT')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')
adata = adata[adata.obs.n_genes_by_counts < 6000, :]
# adata = adata[adata.obs.pct_counts_mt < 20, :]
adata = adata[:,adata.var['mt']==False]
adata.write_h5ad("scatac_heart_674new.h5ad")

In [ ]:
adata = sc.read_h5ad("scatac_heart_674new.h5ad")
adata.var_names_make_unique()

In [ ]:
if issparse(adata.X):
    if not adata.X.has_sorted_indices:
        adata.X.sort_indices()
ro.globalenv["adata"] = adata
adata.obs['n_counts'] = np.array(np.sum(adata.X, axis= 1))

In [ ]:
%%R
seurat_obj = as.Seurat(adata, counts="X", data = NULL)
seurat_obj = RenameAssays(seurat_obj, originalexp = "RNA")
res = SCTransform(object=seurat_obj, vst.flavor = "v2", variable.features.n = 1000 , method = "glmGamPoi", verbose = FALSE)

In [ ]:
gene_list = list(ro.r("rownames(res@assays$SCT@scale.data)"))
norm_x = ro.r("res@assays$SCT@scale.data")
exp_matrix = pd.DataFrame(norm_x, index=gene_list)

adata_new = adata[:,gene_list]
counts = adata_new.X.todense()
seq_depth = adata_new.obs['n_counts'].values

B_cell_result = CSCORE_IRLS(np.array(counts), seq_depth)

p_value = B_cell_result[1]
cor_matrix = (p_value<0.005)*1
print(cor_matrix)

exp_matrix.to_csv(f"./heart_atlas/scatac_heart_674"+"_rna_expression.csv")
cor_matrix = pd.DataFrame(cor_matrix, index = gene_list, columns = gene_list)
cor_matrix.to_csv(f"./heart_atlas/scatac_heart_674"+"_pvalue.csv")